# Annotate incomplete or partially annotated files

IF there are chunks that are not commpletely annotated or have partial annotation run this script as it may try to re-annoate only the parts of the chunk that are not completely annotated. This may require manual cleanup using diffgram and then use this script to see if the A.I can continue to annotate. 

There is no gurantee the A.I will be succesful to annotate.

In [ ]:
!pip install torch transformers diffgram neo4j anthropic pandas tqdm
!pip install llama_index
!pip install boto3

In [ ]:
!pip install arize-phoenix-otel
!pip install openinference-instrumentation-bedrock opentelemetry-exporter-otlp

In [5]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from diffgram import Project
from typing import List, Dict, Optional
import anthropic
import json
from neo4j import GraphDatabase
from tqdm import tqdm
import logging
import os
import sys
import boto3
import requests
import pprint

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to Arize

In [ ]:
# Import the new tracking code
#from bedrock_output_fix import setup_claude_tracking, get_response_with_tracking
from opentelemetry.trace import get_tracer_provider

In [ ]:
from phoenix.otel import register

tracer_provider = register(
  project_name="pre-annotation-with-AI", # Default is 'default'
  endpoint="http://phoenix:6006/v1/traces",
)

In [ ]:
from openinference.instrumentation.bedrock import BedrockInstrumentor
BedrockInstrumentor().instrument(tracer_provider=tracer_provider,
                                capture_response_body=True  # Enable response capture
                                )

In [ ]:
# Use os.getcwd() since __file__ is not available in interactive environments
current_dir = os.getcwd()

# If your structure is such that the package is in the parent directory, compute the parent directory:
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [ ]:
from AgenticWorkflow.bedrock_session import get_boto_session

In [ ]:
session = get_boto_session()

In [ ]:
import  get_claude_kwargs
from get_claude_kwargs import get_response

In [ ]:
def get_response_with_tracking(prompt, job_data = None):
    with get_tracer_provider().get_tracer(__name__).start_as_current_span("claude_request") as span:
        # Convert nested structure to flat attributes with dot notation
        #span.set_attribute("llm.model_name", "anthropic.claude-3-5-sonnet-20240620-v1:0")
        #span.set_attribute("llm.token_count.prompt", len(prompt.split()))
        #span.set_attribute("llm.invocation_parameters", get_response(prompt))
        
        try:
            span.set_attribute("input.value", prompt)
            # Get response using original function
            output = get_response(prompt)
                     
            # Set output as string
            span.set_attribute("output.value", output if output else "None")
        
            span.set_attribute("task.name", job_data['nickname'] if job_data else "None")
            span.set_attribute("task.index", job_data['index'] if job_data else "None")
            span.set_attribute("task.fileID", job_data['file'] if job_data else "None")
            
            # Set span kind as string
            span.set_attribute("openinference.span.kind", "LLM")
            
            return output
            
        except Exception as e:
            # Log error as flat strings
            span.set_attribute("error.message", str(e))
            span.set_attribute("error.type", e.__class__.__name__)
            raise

In [ ]:
prompt= "test if the bedrock connection is established"

In [ ]:
response = get_response_with_tracking(prompt)

## Connect to Diffgram

In [6]:
# Diffgram project configuration
DIFFGRAM_CONFIG = {
    "host": "http://dispatcher:8085",
    "project_string_id": "translucenttracker",
    "client_id": "LIVE__u3v8q0m7tx1p851dp0ap",
    "client_secret": "1qgd8as7xfcbuem6mw9j1z0xvjfmmvlagbugqr8z1g1ntypugr2ul24cce5k"
}

In [7]:
# Initialize connection to Diffgram project
project = Project(host=DIFFGRAM_CONFIG["host"],
        project_string_id = "translucenttracker",
        client_id = "LIVE__u3v8q0m7tx1p851dp0ap",
        client_secret = "1qgd8as7xfcbuem6mw9j1z0xvjfmmvlagbugqr8z1g1ntypugr2ul24cce5k"
      )
project_local = project

## Fetch Schema

In [8]:
# Define and manage NER schema in Diffgram
# Retrieve and process existing schema labels
NER_schema_name = 'ENTITY_TRAINING_SCHEMA'

In [9]:
def get_schema_list(id):
    auth = project.session.auth
    url = f"{DIFFGRAM_CONFIG['host']}/api/project/{DIFFGRAM_CONFIG['project_string_id']}/labels?schema_id={id}"
    # Step 4: Make the POST request using the SDK's session auth
    response = requests.get(url, auth=auth)
    # Step 5: Handle the response
    if response.status_code == 200:
        #print("Annotation update successful!")
        #pprint.pprint(response.json())  # View the updated data
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)  # Print error details for debugging

In [10]:
schema_id = None

# List the existing schemas in your Diffgram project.
schemas = project.schema.list()
schema_list = schemas
print("Existing Schemas in Diffgram:")
print(json.dumps(schemas, indent=2))

# Check if a schema with the name NER_schema_name already exists.
for schema in schemas:
    if schema.get('name') == NER_schema_name:
        schema_id = schema.get('id')
        break

# If the schema does not exist, create a new one.
if schema_id is None:
    print(f"Schema '{NER_schema_name}' not found. Creating a new one...")
    json_response = project.new_schema(name=NER_schema_name)
    schema_id = json_response.get("id")
    print(f"Created new schema with id: {schema_id}")
else:
    print(f"Schema '{NER_schema_name}' already exists with id: {schema_id}")

schema_labels = get_schema_list(schema_id)

# Retrieve existing labels for the schema to avoid duplicates.
schema_label_id_value = []
if schema_labels is not None:
    labels = schema_labels['labels_out']
    for label in labels:
        value = {}
        value['id'] = label['id']
        value['name'] = label['label']['name']
        schema_label_id_value.append(value)

existing_label_names = set()
try:
    schema_label_id_value[0]['name']
    for label in schema_label_id_value:
            label_name = label.get("name")
            if label_name:
                existing_label_names.add(label_name)
    print(existing_label_names)      
except:
     print("There are no schema labels") 

Existing Schemas in Diffgram:
[
  {
    "archived": false,
    "id": 8,
    "is_default": true,
    "member_created_id": 1,
    "member_updated_id": null,
    "name": "Default Schema",
    "project_id": 4,
    "time_created": "2025-02-04 22:16:17",
    "time_updated": null
  },
  {
    "archived": false,
    "id": 9,
    "is_default": false,
    "member_created_id": 10,
    "member_updated_id": null,
    "name": "NER_TRAINING_SCHEMA",
    "project_id": 4,
    "time_created": "2025-02-05 17:08:24",
    "time_updated": null
  },
  {
    "archived": false,
    "id": 11,
    "is_default": false,
    "member_created_id": 10,
    "member_updated_id": null,
    "name": "ENTITY_TRAINING_SCHEMA",
    "project_id": 4,
    "time_created": "2025-02-05 17:20:02",
    "time_updated": null
  }
]
Schema 'ENTITY_TRAINING_SCHEMA' already exists with id: 11
{'O', 'I-SECTION_ID', 'I-DEFINITION', 'B-SEQUENCE_ID', 'I-AUTHORITY', 'I-REQUIREMENT', 'B-METADATA_VALUE', 'I-REGULATION_ID', 'B-SUBSECTION_REF', 'B-

## Diffgram Send Utility

In [11]:
# Functions to send pre-annotations back to Diffgram
# Handle API communication and response processing
def send_preannotation_to_diffgram(file):
    # Step 1: Extract the session's auth credentials (client_id and client_secret)
    auth = project_local.session.auth

    # Step 2: Define the API URL for the custom annotation update endpoint
    file_id = file.id  # Replace with your file ID
    #project_string_id = "your_project_string_id"  # Replace with your project string ID
    url = f"{DIFFGRAM_CONFIG['host']}/api/project/{DIFFGRAM_CONFIG['project_string_id']}/file/{file_id}/annotation/update"

    # Step 3: Define the data (e.g., instance_list) for updating annotations
    data = {
        "instance_list": file.__dict__['instance_list']
    }

    # Step 4: Make the POST request using the SDK's session auth
    response = requests.post(url, json=data, auth=auth)

    # Step 5: Handle the response
    if response.status_code == 200:
        print("Annotation update successful!")
        # print(response.json())  # View the updated data
    else:
        print(f"Error: {response.status_code}")
        print(response.text)  # Print error details for debugging

## NER Validation Prompt

In [12]:
# Create structured prompt for NER tagging
# Include rules and formatting guidelines for entity recognition
def create_ner_prompt(text: str, schema: List[Dict]) -> str:
    """
    Creates a prompt for NER tagging that defaults to O tag for most text.
    """
    schema_tags = sorted(set(s['name'] for s in schema))
    
    prompt = """You are a Named Entity Recognition system for legislative text.
Your task is to label each word using only the following allowed tags:
{allowed_tags}

Rules:
1. Default to 'O' tag - most words should be tagged as Outside
2. Only tag these specific elements:
   - Section references (e.g., "section 46" -> B-SECTION_REF, I-SECTION_REF)
   - Subsection references (e.g., "(2)", "(a)" -> B-SUBSECTION_REF)
   - Essential metadata fields and their values
3. Tag principles:
   - All punctuation should be 'O'
   - Use B- prefix only for start of key references
   - Use I- prefix for continuing words of same reference
   - When in doubt, use 'O' tag

Each space should be annotated separately. For e.g. If you see child 's annotate it as child and 's as two different words as there is a sapce between them. All ' s must be treated as one word 's.
If there are no spaces between words like Act-Motor or 2subsection or 1in or b or ill. . treat this as one word. If b. and c. are separatd by a space then b. is one word and c. is another word. If there is a space between say en . then en is one word and the . is another word which should have its own label.

Output format:
[
    {{"word": "section", "tag": "B-SECTION_REF"}},
    {{"word": "46", "tag": "I-SECTION_REF"}},
    {{"word": ",", "tag": "O"}},
    {{"word": "subsection", "tag": "O"}},
    {{"word": "(", "tag": "O"}},
    {{"word": "2", "tag": "B-SUBSECTION_REF"}},
    {{"word": "'s", "tag": "O"}},
    {{"word": ")", "tag": "O"}}
]

Text to label:
{text}

Provide only the JSON output with no additional explanation."""

    tags_str = "\n".join(f"- {tag}" for tag in schema_tags)
    return prompt.format(allowed_tags=tags_str, text=text)

## Utilities

In [13]:
def extract_word_data(url):
    # Original URL with localhost
    # Replace localhost with ngrok URL (example: "https://example.ngrok.io")
    file_url = url.replace("http://localhost:8085", DIFFGRAM_CONFIG['host'])

    # Make the GET request to fetch the file
    response = requests.get(file_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON content into a Python dictionary
        data = response.json()  # Assuming the file is in JSON format
        return data
    else:
        print(f"Failed to retrieve the file. Status code: {response.status_code}")

In [14]:
# Utility functions for processing Diffgram annotations
# Extract and format word-level data from files
def get_file_number(completed_annotations, files_index_in_job):
    data = []
    sentences = []
    labels = []
    data_index = 0
    for completed_annotation in completed_annotations:
        try:
            file_index = int(completed_annotation)
            files_index_in_job.append(completed_annotation)
            #print(completed_annotation)
            continue
        except:
            #print(f"{completed_annotation} is not a file")
            continue
            
        #print(f"{completed_annotation} ----")
        if (completed_annotation != 'attribute_groups_reference')  \
            and (completed_annotation != 'export_info') \
            and (completed_annotation != 'label_map') \
            and (completed_annotation != 'readme') \
            and (completed_annotation != 'label_colour_map'):
            sentence_local = []
            labels_local = []

            # First get the point where the annotation is started
            for start in completed_annotations[completed_annotation]['instance_list']:
                if 'start_token' in start:
                    start_token =  start['start_token']
                    break

            #start_token = completed_annotations[completed_annotation]['instance_list'][0]['start_token']
            for annotated_index in range(start_token, len(completed_annotations[completed_annotation]['text']['tokens']['words'])):
                # check if this text is annotated
                for data in completed_annotations[completed_annotation]['instance_list']:
                    if 'start_token' in data:
                        if annotated_index == data['start_token']:
                            sentence_local.append(completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value'])
                            labels_local.append(completed_annotations['label_map'][str(data['label_file_id'])])
                            #print(f"{completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value']} - {completed_annotations['label_map'][str(data['label_file_id'])]}")
                            break;
            sentences.append(sentence_local)       
            labels.append(labels_local)
            data_index+=1

## Capture incomplete annotation

In [15]:
results = project_local.job
get_job = project_local.job.list(limit=10000, page_number=1)

In [16]:
jobs_with_data_index = []
for job_key, job_list in enumerate(get_job):
    try:
        nickname = job_list['attached_directories_dict']['attached_directories_list'][0]['nickname']
        if nickname:
            job_value = {}
            job_value['nickname'] = nickname
            job_value['index'] = job_key
            jobs_with_data_index.append(job_value)
        #print(nickname)
    except KeyError:
        print("Key not found.")
    except IndexError:
        print("List index out of range.")

In [17]:
def vallidate_annotation(completed_annotations, files_that_need_annotation, failed_to_annotate, annotation_not_complete, job_index):
    for completed_annotation in completed_annotations:
        #print(f"{completed_annotation} ----")
        if (completed_annotation != 'attribute_groups_reference')  \
            and (completed_annotation != 'export_info') \
            and (completed_annotation != 'label_map') \
            and (completed_annotation != 'readme') \
            and (completed_annotation != 'label_colour_map'):
            file = project_local.file.get_by_id(completed_annotation,with_instances=True)
            url = file.__dict__['text']['tokens_url_signed']
            data = extract_word_data(url)
            word_count = 0
            for word in data['nltk']['words']:
                if (word['value'] == '\n'):
                    continue
                word_count += 1
            annotated_count = len(completed_annotations[completed_annotation]['instance_list'])      
            if (word_count == annotated_count):
                continue
                #print(f"SUCCESS: The file id is: {completed_annotation} and total annotation is {annotated_count} and word count is {word_count}")
            else:
                job_value = {}
                job_value['nickname'] = job_index['nickname']
                job_value['index'] = job_index['index']
                job_value['file'] = completed_annotation
                jobs_with_data_index.append(job_value)
                if (annotated_count) == 0:
                    failed_to_annotate.append(job_value)
                else:
                    annotation_not_complete.append(job_value)
                print(f"ERROR: The file id is: {completed_annotation} and total annotation is {annotated_count} and word count is {word_count}")
                files_that_need_annotation.append(completed_annotation)

In [ ]:
files_that_need_annotation = []
failed_to_annotate = []
annotation_not_complete = []
for job_index in jobs_with_data_index[0:]:
    print(f"The job nickname is {job_index['nickname']} and the index is {job_index['index']}")
    results.refresh_from_dict(get_job[job_index['index']])
    completed_annotations = results.generate_export()
    vallidate_annotation(completed_annotations,files_that_need_annotation, failed_to_annotate, annotation_not_complete, job_index)
print(len(files_that_need_annotation))   

## Process incomplete annotation

In [ ]:
annotation_not_complete

In [ ]:
def normalize_string(s):
    # Handle double backticks first
    s = s.replace('``', '"')
    # Handle other quote variations
    s = s.replace('"', '"').replace('"', '"').replace('\'\'', '"')
    s = s.replace('\\', '')
    s = s.replace('\'s', 'is')
    return s

In [324]:
import math
def process_per_file_annotation(job_data):
    try:
        file_id = job_data['file']
        file = project_local.file.get_by_id(file_id, with_instances=True)
        data = extract_word_data(file.__dict__['text']['tokens_url_signed'])
        
        # Get current annotation count
        num_annotated_text_index = len(file.__dict__['instance_list']) - 1
        
        # Count actual words (excluding newlines)
        word_count = sum(1 for word in data['nltk']['words'] if word['value'] != '\n')
        
        # Skip if only few words remaining
        if abs(word_count - num_annotated_text_index) < 4:
            print(f"Skipping file {file_id}: only {abs(word_count - num_annotated_text_index)} words remaining")
            return {
                'status': 'success',
                'job_index': job_data['index'],
                'nickname': job_data['nickname'],
                'file_id': job_data['file']
            }

        # Get remaining words for annotation
        remaining_words = []
        word_index = 0
        actual_indices = []  # Track actual indices of remaining words
        
        for i, word in enumerate(data['nltk']['words']):
            if word['value'] != '\n':
                if word_index >= num_annotated_text_index:
                    remaining_words.append(word['value'])
                    actual_indices.append(i)
                word_index += 1

        # Create prompt for remaining words
        words = "\n".join(remaining_words)
        prompt = create_ner_prompt(words, schema_label_id_value)
        print(prompt)
        #return
        response = get_response_with_tracking(prompt, job_data)
        ai_annotation = json.loads(response)

        # Process annotations
        instance_list = file.__dict__['instance_list']
        for i, (word_data, actual_idx) in enumerate(zip(ai_annotation, actual_indices)):
            if normalize_string(data['nltk']['words'][actual_idx]['value']) == normalize_string(word_data['word']):
                label_id = next((label['id'] for label in schema_label_id_value 
                               if label['name'] == word_data['tag']), None)
                
                if label_id:
                    instance = {
                        "label_file_id": label_id,
                        "start_token": actual_idx,  # Use actual index from tokens
                        "end_token": actual_idx,    # Use actual index from tokens
                        "type": 'text_token'
                    }
                    instance_list.append(instance)

        # Update and send annotations
        file.__dict__['instance_list'] = instance_list
        sent = send_preannotation_to_diffgram(file)
        
        return {
            'status': 'success',
            'job_index': job_data['index'],
            'nickname': job_data['nickname'],
            'file_id': job_data['file']
        }
        
    except Exception as e:
        print(f"Error in job {job_data['nickname']}: {str(e)}")
        return {
            'status': 'error',
            'job_index': job_data['index'],
            'nickname': job_data['nickname'],
            'error': str(e)
        }

## Parallel process

In [313]:
from concurrent.futures import ThreadPoolExecutor

def process_failed_annotations(failed_files, max_workers=4):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all files for processing
        futures = [
            executor.submit(process_per_file_annotation, file)
            for file in failed_files
        ]
        
        # Track results
        processed_files = []
        for future in futures:
            try:
                result = future.result()
                processed_files.append(result)
                
                # Assuming process_per_file_annotation returns a dict with status
                if result.get('status') == 'success':
                    print(f"Successfully processed file {result.get('file_id')}")
                else:
                    print(f"Failed to process file {result.get('file_id')}: {result.get('error')}")
                    
            except Exception as e:
                print(f"Exception in worker thread: {str(e)}")
                
        return processed_files

In [271]:
annotation_not_complete = []

In [ ]:
def get_file_annotations(file_id, project_local):
    url = f"{DIFFGRAM_CONFIG['host']}/api/project/{DIFFGRAM_CONFIG['project_string_id']}/file/{file_id}/annotation/list"
    auth = project_local.session.auth
    
    data = {"directory_id":197,
            "job_id":'null',
            "task_child_file_id":20723}
    
    response = requests.post(
        url, 
        json = data,
        auth=auth
    )
    return resposnse.json()

In [319]:
annotation_not_complete = [
 {'nickname': 'NER_train_batch_218', 'index': 31, 'file': '19747'},
 {'nickname': 'NER_train_batch_213', 'index': 36, 'file': '19573'},
 {'nickname': 'NER_train_batch_209', 'index': 40, 'file': '19465'},
 {'nickname': 'NER_train_batch_202', 'index': 47, 'file': '19234'},
 {'nickname': 'NER_train_batch_193', 'index': 56, 'file': '18936'},
 {'nickname': 'NER_train_batch_167', 'index': 82, 'file': '18110'},
 {'nickname': 'NER_train_batch_158', 'index': 91, 'file': '17842'},
 {'nickname': 'NER_train_batch_153', 'index': 96, 'file': '17653'},
 {'nickname': 'NER_train_batch_149', 'index': 100, 'file': '17554'},
 {'nickname': 'NER_train_batch_136', 'index': 113, 'file': '17113'},
 {'nickname': 'NER_train_batch_135', 'index': 114, 'file': '17084'},
 {'nickname': 'NER_train_batch_107', 'index': 142, 'file': '16203'},
 {'nickname': 'NER_train_batch_101', 'index': 148, 'file': '15999'},
 {'nickname': 'NER_train_batch_97', 'index': 152, 'file': '15889'},
 {'nickname': 'NER_train_batch_89', 'index': 160, 'file': '15628'},
 {'nickname': 'NER_train_batch_83', 'index': 166, 'file': '15434'},
 {'nickname': 'NER_train_batch_75', 'index': 174, 'file': '15162'},
 {'nickname': 'NER_train_batch_59', 'index': 190, 'file': '14658'},
 {'nickname': 'NER_train_batch_57', 'index': 192, 'file': '14588'},
 {'nickname': 'NER_train_batch_51', 'index': 198, 'file': '14410'},
 {'nickname': 'NER_train_batch_42', 'index': 207, 'file': '14111'},
 {'nickname': 'NER_train_batch_28', 'index': 221, 'file': '13669'},
 {'nickname': 'NER_train_batch_22', 'index': 227, 'file': '13480'},
 {'nickname': 'NER_train_batch_16', 'index': 233, 'file': '13298'},
 {'nickname': 'NER_train_batch_5', 'index': 244, 'file': '12875'},]

In [337]:
#for jobs_data in annotation_not_complete[3:]:
#    print(job_data)
annotation_not_complete = [{'nickname': 'NER_train_batch_153', 'index': 96, 'file': '17653'}]
results = process_failed_annotations(annotation_not_complete, max_workers=10)
#process_per_file_annotation(annotation_not_complete[2])

You are a Named Entity Recognition system for legislative text.
Your task is to label each word using only the following allowed tags:
- B-ACT_ID
- B-ACT_NAME
- B-AUTHORITY
- B-CHUNK_ID
- B-DEFINITION
- B-METADATA_FIELD
- B-METADATA_VALUE
- B-REGULATION_ID
- B-REQUIREMENT
- B-SECTION_ID
- B-SECTION_NAME
- B-SECTION_REF
- B-SEQUENCE_ID
- B-SUBSECTION_REF
- I-ACT_ID
- I-ACT_NAME
- I-AUTHORITY
- I-CHUNK_ID
- I-DEFINITION
- I-METADATA_FIELD
- I-METADATA_VALUE
- I-REGULATION_ID
- I-REQUIREMENT
- I-SECTION_ID
- I-SECTION_NAME
- I-SECTION_REF
- I-SEQUENCE_ID
- I-SUBSECTION_REF
- O

Rules:
1. Default to 'O' tag - most words should be tagged as Outside
2. Only tag these specific elements:
   - Section references (e.g., "section 46" -> B-SECTION_REF, I-SECTION_REF)
   - Subsection references (e.g., "(2)", "(a)" -> B-SUBSECTION_REF)
   - Essential metadata fields and their values
3. Tag principles:
   - All punctuation should be 'O'
   - Use B- prefix only for start of key references
   - Use I- 